In [39]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [40]:
#reading traing data
#usecols: select the columns to include, drop the column named id
# dtype: specify the type of the data for certain series
# converters: can define a lambda function, some unit_sales is negative, so convert that to 0 if negative
# nrows: specify the number of rows to read in
# skiprows: skip over this range of rows

df_train = pd.read_csv(
    'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)
df_train.tail()

,date,store_nbr,item_nbr,unit_sales,onpromotion
59038127,2017-08-15,54,2089339,1.609438,False
59038128,2017-08-15,54,2106464,0.693147,True
59038129,2017-08-15,54,2110456,5.262690,False
59038130,2017-08-15,54,2113914,5.293305,True
59038131,2017-08-15,54,2116416,1.098612,False


In [41]:
# only take training data for year of 2017
df_2017=df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [42]:
# free momory
del df_train

In [43]:
# Fill gaps in dates - for example a product has no report in certain store for Jan 2018, but has report
#    for other store in Jan 2018, an entry will be created for that product-store for Jan 2018
#    with value np.nan (NaN)
df_2017_reindex=df_2017.set_index(["store_nbr","item_nbr","date"])
df_2017_reindex.head()

unit_sales onpromotion
store_nbr item_nbr date                              
25        99197    2017-01-01    0.693147       False
          103665   2017-01-01    2.079442       False
          105574   2017-01-01    0.693147       False
          105857   2017-01-01    1.609438       False
          106716   2017-01-01    1.098612       False

In [44]:
# Reading test.csv data
# Same structure as
df_test=pd.read_csv('test.csv',usecols=[0,1,2,3,4],
                   dtype={'onpromotion':bool},
                    parse_dates=["date"]).set_index(['store_nbr','item_nbr', 'date'])
df_test.head()                          

id onpromotion
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040       False
          99197    2017-08-16  125497041       False
          103501   2017-08-16  125497042       False
          103520   2017-08-16  125497043       False
          103665   2017-08-16  125497044       False

In [45]:
#Read supplementary files items, stores
items=pd.read_csv('items.csv').set_index('item_nbr')
stores=pd.read_csv('stores.csv').set_index('store_nbr')

In [46]:
items.head() 

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [47]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [48]:
#Encoding strings to unique numbers
items['family']=LabelEncoder().fit_transform(items['family'].values)

stores['city'] = LabelEncoder().fit_transform(stores['city'].values)
stores['state'] = LabelEncoder().fit_transform(stores['state'].values)
stores['type'] = LabelEncoder().fit_transform(stores['type'].values)

In [49]:
items.head()

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103501,7,3008,0
103520,12,1028,0
103665,5,2712,1


In [50]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
2,18,12,3,13
3,18,12,3,8
4,18,12,3,9
5,21,14,3,4


In [51]:
# onpromotion columns in training datasets
promo_2017_train=df_2017_reindex[['onpromotion']]
promo_2017_train.head()

onpromotion
store_nbr item_nbr date                  
25        99197    2017-01-01       False
          103665   2017-01-01       False
          105574   2017-01-01       False
          105857   2017-01-01       False
          106716   2017-01-01       False

In [52]:
# move date index to column lables
#fill nan values with 'False'
promo_2017_train_unst=promo_2017_train.unstack(level=-1).fillna(False)
promo_2017_train_unst.head()

onpromotion                                              \
date                2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False       True      False      False   

                                                                           \
date               2017-01-06 2017-01-07 2017-01-08 2017-01-09 2017-01-10   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574         True      False      False      False      False   

                      ...                                                  \
date                  ...     2017-08-06 2017-08-07 2017-08-08 2017-08-09   
store_nbr item_nbr    ...                                                   
1         96995       ...          False      False      False      False   
          99197       ...          False      False      False      False   
          103520      ...          False      False      False      False   
          103665      ...          False      False      False      False   
          105574      ...          False      False      False      False   

                                                                           \
date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                               
date               2017-08-15  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 227 columns]

In [53]:
#Take dates of training datasets,replace the column labels with date
promo_2017_train_unst.columns=promo_2017_train_unst.columns.get_level_values(-1)
promo_2017_train_unst.head()

date               2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False       True      False      False   

date               2017-01-06 2017-01-07 2017-01-08 2017-01-09 2017-01-10  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574         True      False      False      False      False   

date                  ...     2017-08-06 2017-08-07 2017-08-08 2017-08-09  \
store_nbr item_nbr    ...                                                   
1         96995       ...          False      False      False      False   
          99197       ...          False      False      False      False   
          103520      ...          False      False      False      False   
          103665      ...          False      False      False      False   
          105574      ...          False      False      False      False   

date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

date               2017-08-15  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 227 columns]

In [54]:
# promotion data in test dataset
promo_2017_test=df_test[["onpromotion"]]
promo_2017_test.head()

onpromotion
store_nbr item_nbr date                  
1         96995    2017-08-16       False
          99197    2017-08-16       False
          103501   2017-08-16       False
          103520   2017-08-16       False
          103665   2017-08-16       False

In [55]:
promo_2017_test_unst=promo_2017_test.unstack(level=-1).fillna(False)
promo_2017_test_unst.head()

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

In [56]:
# assighn column lables with dates
promo_2017_test_unst.columns=promo_2017_test_unst.columns.get_level_values(1)

In [57]:
promo_2017_test_unst.head()

date               2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

In [58]:
#Replace promo_2017_test index with the index in promo_2017_train, so train and test dataset can merge togeher later.
#If there's a new index and reindex the dataframe.by default values in the new index that do not have corresponding records in the dataframe are assigned NaN
# NaN filled with False
promo_2017_test_unst=promo_2017_test_unst.reindex(promo_2017_train_unst.index).fillna(False)

In [59]:
promo_2017_test_unst.head()

date               2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False

In [60]:
#generate promotion information from 2017-1-1 to 2017-8-31
promo_2017=pd.concat([promo_2017_train_unst,promo_2017_test_unst],axis=1)
promo_2017.head()

date               2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False       True      False      False   

date               2017-01-06 2017-01-07 2017-01-08 2017-01-09 2017-01-10  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574         True      False      False      False      False   

date                  ...     2017-08-22 2017-08-23 2017-08-24 2017-08-25  \
store_nbr item_nbr    ...                                                   
1         96995       ...          False      False      False      False   
          99197       ...          False      False      False      False   
          103520      ...          False      False      False      False   
          103665      ...          False      False      False      False   
          105574      ...          False      False      False      False   

date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30  \
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 243 columns]

In [61]:
del promo_2017_train_unst,promo_2017_test_unst,promo_2017_test,promo_2017_train

In [62]:
#prepare dataframe of unit sales informaion: colume lables=dates
df_2017_sales=df_2017_reindex[['unit_sales']].unstack(level=-1).fillna(0)
df_2017_sales.columns=df_2017_sales.columns.get_level_values(1)
df_2017_sales.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [63]:
#reindex items and stores to prepare to adjust with df_2017_sales datframe
items = items.reindex(df_2017_sales.index.get_level_values(1))
stores = stores.reindex(df_2017_sales.index.get_level_values(0))

In [64]:
items.head()

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103520,12,1028,0
103665,5,2712,1
105574,12,1045,0


items.shape

In [65]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13


In [66]:
#define a function
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt-timedelta(days=minus), periods=periods, freq=freq)]
def prepare_dataset(df, promo_df, t2017, is_train=True):
    X=pd.DataFrame({
        'family': items['family'].values,
        'class': items['class'].values,
        'perishable': items['perishable'].values,
        
        'day_1_2017': get_timespan(df_2017_sales, t2017, 1, 1).values.ravel(), # the sales at the date t2017 
        'mean_3_2017': get_timespan(df_2017_sales, t2017, 3, 3).mean(axis=1).values, # the mean sales during 3 days of periods before t2017
        'mean_7_2017': get_timespan(df_2017_sales, t2017, 7, 7).mean(axis=1).values, # the mean sales during 1 week of periods before t2017
        'mean_14_2017': get_timespan(df_2017_sales, t2017, 14, 14).mean(axis=1).values,# the mean sales during 2 weeks of periods before t2017
        'mean_28_2017': get_timespan(df_2017_sales, t2017, 28, 28).mean(axis=1).values, # the mean sales during 4 weeks of periods before t2017
        'mean_63_2017': get_timespan(df_2017_sales, t2017, 63, 63).mean(axis=1).values, # the mean sales during 3 months of periods before t2017
        'mean_140_2017': get_timespan(df_2017_sales, t2017, 140, 140).mean(axis=1).values, # the mean sales during 140 days of periods before t2017
        
        "median_3_2017": get_timespan(df_2017_sales, t2017, 3, 3).median(axis=1).values, # the median values of sales during 3 days of periods before t2017
        "median_7_2017": get_timespan(df_2017_sales, t2017, 7, 7).median(axis=1).values,  # the median values of sales during 1 week  of periods before t2017
        "median_14_2017": get_timespan(df_2017_sales, t2017, 14, 14).median(axis=1).values, # the median values of sales during 2 week  of periods before t2017
        "median_28_2017": get_timespan(df_2017_sales, t2017, 28, 28).median(axis=1).values, # the median values of sales during 4 week  of periods before t2017
        "median_63_2017": get_timespan(df_2017_sales, t2017, 63, 63).median(axis=1).values, # the median values of sales during 3 months of periods before t2017
        "median_140_2017": get_timespan(df_2017_sales, t2017, 140, 140).median(axis=1).values, # the median values of sales during 140 days of periods before t2017

        "promo_14_2017":get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values, #total promotion for 14 dyas before the date of t2017 
        "promo_63_2017":get_timespan(promo_2017, t2017, 63, 63).sum(axis=1).values, #total promotion for 60 dyas before the date of t2017
        "promo_140_2017":get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values, #total promotion for 140 dyas before the date of t2017
    })
    
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 28-i, 4, freq='7D').mean(axis=1).values # Mean sales every day of week in a month (e.g.,mean sales in every Wednesday, Thursday,...Tuesday in May)
        X['min_4_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 28-i, 4, freq='7D').min(axis=1).values # Minium sales every day of week in a month (e.g.,minimum sales in every Wednesday, Thursday,...Tuesday in May)
        X['max_4_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 28-i, 4, freq='7D').max(axis=1).values # Maximum sales every day of week in a month (e.g.,maximum sales in every Wednesday, Thursday,...Tuesday in May)
        X['mean_20_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 140-i, 20, freq='7D').mean(axis=1).values # Mean sales every day of week in 5 months(140 days) (e.g.,mean sales in every Wednesday, Thursday,...Tuesday from January to May)
        X['min_20_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 140-i, 20, freq='7D').min(axis=1).values  # Minimum sales every day of week in 5 months (140 days) (e.g.minimum sales in every Wednesday, Thursday,...Tuesday from January to May)
        X['max_20_dow{}_2017'.format(i)]=get_timespan(df_2017_sales, t2017, 140-i, 20, freq='7D').max(axis=1).values # Maximum sales every day of week in 5 months (140 days) (e.g.,maximum sales in every Wednesday, Thursday,...Tuesday from January to May)
    
    for i in range(16):
        X['promo_{}'.format(i)]=promo_2017[t2017+timedelta(days=i)].values.astype(np.uint8) #promotion info for the 16 days period after the t2017      
    if is_train:
        y=df_2017_sales[pd.date_range(t2017,periods=16)].values # output values for training, 16 columns for 16 days
        return X,y
    return X

In [67]:
print('Preparing dataset....')
t2017=date(2017, 6, 14)
num_days=6

#preparing the training data
X_l,y_l=[],[]
for i in range(num_days):
    X_tmp,y_tmp=prepare_dataset(df_2017_sales,promo_2017,t2017+timedelta(days=7*i)) # t2017= include the dates every week after the date of 06/14/2017 for 5 weeks.
    X_l.append(X_tmp)
    y_l.append(y_tmp)   
X_train=pd.concat(X_l,axis=0) 
y_train=np.concatenate(y_l,axis=0) #Every (repeated 6 iteration) matrix of X_train is corresponde to the y_train, which is 16 days of unit_sales after t2017

Preparing dataset....


In [68]:
del X_l, y_l

In [69]:
#Validation set unit sales from 2017-7-26 to 2017-8-15
X_val,y_val=prepare_dataset(df_2017_sales, promo_2017,date(2017,7,26))

In [70]:
X_test = prepare_dataset(df_2017_sales, promo_2017, date(2017, 8, 16), is_train=False)

In [71]:
#Set up the lgbm model 
print("Training and predicting models...")
params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    #validation dataset from train.csv predicted by trained model
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    #Test dataset from test.csv predicted by trained model
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

Training and predicting models...
Step 1


C:\Users\ooppa\Anaconda3\lib\site-packages\lightgbm\basic.py:1005: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.317148	valid_1's l2: 0.312638
[200]	training's l2: 0.293726	valid_1's l2: 0.292204
[300]	training's l2: 0.289278	valid_1's l2: 0.289434
[400]	training's l2: 0.286857	valid_1's l2: 0.288519
[500]	training's l2: 0.285114	valid_1's l2: 0.288081
[600]	training's l2: 0.283586	valid_1's l2: 0.287803
[700]	training's l2: 0.282223	valid_1's l2: 0.287666
[800]	training's l2: 0.280913	valid_1's l2: 0.287497
[900]	training's l2: 0.279686	valid_1's l2: 0.287388
[1000]	training's l2: 0.27851	valid_1's l2: 0.287291
[1100]	training's l2: 0.277359	valid_1's l2: 0.287254
[1200]	training's l2: 0.276266	valid_1's l2: 0.287199
[1300]	training's l2: 0.275199	valid_1's l2: 0.28716
[1400]	training's l2: 0.274156	valid_1's l2: 0.287105
[1500]	training's l2: 0.273139	valid_1's l2: 0.287064
[1600]	training's l2: 0.272133	valid_1's l2: 0.287005
Early stopping, best iteration is:
[1612]	training's l2: 0.27201	valid_1's l2: 0.2869

In [72]:
# Calculate the errors
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

Validation mse: 0.355367573278
nwrmsle = 0.5956632835286029


In [76]:
# arrange the dataframe of test results
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017_sales.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales") 


In [77]:
# Prepare the submission file
print("Making submission...")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Making submission...


In [78]:
df_test.shape

(3370464, 2)